# CLEAR VHEE experiment calculations

Angle calculations for the water tank irradiation experiment at CLEAR.

In [1]:
import numpy as np

In [13]:
def getAngleAndSize(beta_entrance, beta_centre, emittance, distance):
    '''
    Calculate the angle of the beam cone

    Inputs:
    - beta_entrance:  beta at the beginning of the box
    - beta_centre:    beta at the waist 
    - emittance
    - distance:       distance between the two points where the betas are calculated

    Outputs:
    - angle:          the beam angle
    - size:           the beam minimum size

    Last modified: 10.09.2018 by Eugenio Senes
    '''
    angle = np.degrees(np.arctan(np.sqrt(emittance*(beta_entrance-beta_centre))/distance))
    size = 2*np.sqrt(beta_centre*emittance)
    return angle, size


In [17]:
getAngleAndSize(20, 1.125e-3, 15e-6, 0.15)

(6.586591925812557, 0.0002598076211353316)